In [1]:
import tensorflow as tf
from tensorflow import keras

from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from numpy import argmax, array_equal
import matplotlib.pyplot as plt
from keras.models import Model
from imgaug import augmenters
from random import randint
import pandas as pd
import numpy as np
import seaborn as sns

Using TensorFlow backend.


In [2]:
import missingno as msno
%matplotlib inline

In [3]:
import sklearn.linear_model as lm

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve, auc, f1_score, precision_score, recall_score
from sklearn.svm import SVC

In [5]:
data = pd.read_csv("C:/Users/user/Desktop/Anaconda33/bank-additional-full222.csv")
data.head()

,age,job,marital,education,default,balance,housing,loan,y
0,59,admin.,married,secondary,no,2343,yes,no,yes
1,56,admin.,married,secondary,no,45,no,no,yes
2,41,technician,married,secondary,no,1270,yes,no,yes
3,55,services,married,secondary,no,2476,yes,no,yes
4,54,admin.,married,tertiary,no,184,no,no,yes


In [6]:
print(data.isnull().sum())

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
y            0
dtype: int64


In [ ]:
numeric_features = data.select_dtypes(include=[np.number])

numeric_features.columns

In [ ]:
categorical_features = data.select_dtypes(include=[np.object])
categorical_features.columns

In [ ]:
data.describe()

In [ ]:
data1 = data[data['y'] == 'yes']
data2 = data[data['y'] == 'no']

In [ ]:
data['y'].value_counts()

In [ ]:
numeric_df = data.select_dtypes(exclude="object")

In [ ]:
corr_numeric = numeric_df.corr()

sns.heatmap(corr_numeric, cbar=True, cmap="YlGnBu")
plt.title("Correlation Matrix", fontsize=16)
plt.show()

In [14]:
df1 = data[['age','balance','y']]
sns.pairplot(df1, kind="scatter",hue="y",palette="Set2")
plt.show()

In [ ]:
fig1, ax = plt.subplots(1, 2, figsize=(12,14))

a1=ax[0].bar(data1['job'].unique(),height = data1['job'].value_counts(),color='#3cb44b')
a2=ax[0].bar(data1['job'].unique(),height = data2['job'].value_counts()[data1['job'].value_counts().index],bottom = data1['job'].value_counts(),color = '#aaffc3') 
ax[0].title.set_text('Type of Job')
ax[0].tick_params(axis='x',rotation=90)


ax[1].bar(data1['housing'].unique(),height = data1['housing'].value_counts(),color='#3cb44b')
ax[1].bar(data1['housing'].unique(),height = data2['housing'].value_counts()[data1['housing'].value_counts().index],bottom = data1['housing'].value_counts(),color = '#aaffc3') 
ax[1].title.set_text('Has housing loan')


plt.figlegend((a1[0], a2[0]), ('Yes', 'No'),loc="right",title = "Term deposit")
plt.show()


In [ ]:
fig2, ax = plt.subplots(1, 2, figsize=(12,14))

a1=ax[0].bar(data1['default'].unique(),height = data1['default'].value_counts(),color='#3cb44b')
a2=ax[0].bar(data1['default'].unique(),height = data2['default'].value_counts()[data1['default'].value_counts().index],bottom = data1['default'].value_counts(),color = '#aaffc3') 
ax[0].title.set_text('Has credit in default')
ax[0].tick_params(axis='x',rotation=90)


ax[1].bar(data1['education'].unique(),height = data1['education'].value_counts(),color='#3cb44b') #row=0, col=1
ax[1].bar(data1['education'].unique(),height = data2['education'].value_counts()[data1['education'].value_counts().index],bottom = data1['education'].value_counts(),color = '#aaffc3') 
ax[1].title.set_text('Education')
ax[1].tick_params(axis='x',rotation=90)

plt.figlegend((a1[0], a2[0]), ('Yes', 'No'),loc="right",title = "Term deposit")
plt.show()


In [12]:
predictors = data.iloc[:,0:8]
predictors1=data[['age', 'balance']]
y = data.iloc[:,8]
X = pd.get_dummies(predictors) #One hot encoder
X1=predictors1

In [ ]:
y.head()

In [ ]:
X.head()

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = lm.LogisticRegression(random_state=0, solver='lbfgs',multi_class='auto',max_iter=1000).fit(X_train,y_train)
y_pred = model.predict_proba(X_test)
y_pred = y_pred[:,1]
y_test1=y_test
y_test = label_binarize(y_test,classes=['no','yes'])
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred)
roc_auc_lr = auc(fpr_lr, tpr_lr)
y_pred = model.predict(X_test)
y_pred1=y_pred
y_pred = label_binarize(y_pred,classes=['no','yes'])
print("Precision: ",round(precision_score(y_test,y_pred),2),"Recall: ",round(recall_score(y_test,y_pred),2))
print("AUC: ",roc_auc_lr)
mat = confusion_matrix(y_test1,y_pred1,labels=['no','yes'])
print(mat)

NameError: name 'X' is not defined

In [ ]:
import statsmodels.api as sm
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3)
y_train = label_binarize(y_train,classes=['no','yes'])
logit_model=sm.OLS(y_train,X_train)
result=logit_model.fit()
print(result.summary())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
tree = DecisionTreeClassifier(criterion="entropy", max_depth=7)
model = tree.fit(X_train,y_train)
y_pred = model.predict(X_test)
y_test1=y_test
y_pred1=y_pred
y_test = label_binarize(y_test,classes=['no','yes'])
y_pred = label_binarize(y_pred,classes=['no','yes'])
fpr_dr, tpr_dr, _ = roc_curve(y_test, y_pred)
roc_auc_dr = auc(fpr_dr, tpr_dr)
print("Precision: ",round(precision_score(y_test,y_pred),2),"Recall: ",round(recall_score(y_test,y_pred),2))
print("AUC: ",roc_auc_dr)
mat = confusion_matrix(y_test1,y_pred1,labels=['no','yes'])
print(mat)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
forest = RandomForestClassifier(n_estimators= 1000,criterion="gini", max_depth=5,min_samples_split = 0.4,min_samples_leaf=1, class_weight="balanced")
model = forest.fit(X_train,y_train)
y_pred = model.predict(X_test)
pd.Series(y_pred).value_counts()
y_test1=y_test
y_pred1=y_pred
y_test = label_binarize(y_test,classes=['no','yes'])
y_pred = label_binarize(y_pred,classes=['no','yes'])
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred)
roc_auc_rf = auc(fpr_rf, tpr_rf)
print("Precision: ",round(precision_score(y_test,y_pred),2),"Recall: ",round(recall_score(y_test,y_pred),2))
print("AUC: ",roc_auc_rf)
mat = confusion_matrix(y_test1,y_pred1,labels=['no','yes'])
print(mat)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
knn_clf = KNeighborsClassifier(n_neighbors=5)
model=knn_clf.fit(X_train,y_train)
y_pred = model.predict(X_test)
pd.Series(y_pred).value_counts()
y_test1=y_test
y_pred1=y_pred
y_test = label_binarize(y_test,classes=['no','yes'])
y_pred = label_binarize(y_pred,classes=['no','yes'])
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_pred)
roc_auc_knn = auc(fpr_knn, tpr_knn)
print("Precision: ",round(precision_score(y_test,y_pred),2),"Recall: ",round(recall_score(y_test,y_pred),2))
print("AUC: ",roc_auc_knn)
mat = confusion_matrix(y_test1,y_pred1,labels=['no','yes'])
print(mat)

In [ ]:
plt.figure()
lw=2

plt.plot(fpr_lr, tpr_lr,
         label='Logistic regression ROC curve (area = {0:0.4f})'
               ''.format(roc_auc_lr),
         color='deeppink', linestyle=':', linewidth=2)

plt.plot(fpr_rf, tpr_rf,
         label='Random forest ROC curve (area = {0:0.4f})'
               ''.format(roc_auc_rf),
         color='blue', linestyle=':', linewidth=2)

plt.plot(fpr_knn, tpr_knn,
         label='KNN ROC curve (area = {0:0.4f})'
               ''.format(roc_auc_knn),
         color='red', linestyle=':', linewidth=2)
         
plt.plot(fpr_dr, tpr_dr,
         label='Decision tree ROC curve (area = {0:0.4f})'
               ''.format(roc_auc_dr),
         color='green', linestyle=':', linewidth=2)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.00])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')

plt.legend(bbox_to_anchor=(1, 0.5))

#plt.legend(loc="lower right")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
perp_model = lm.Perceptron().fit(X_train_std,y_train)
y_pred = perp_model.predict(X_test_std)
print("Accuracy: ",round(accuracy_score(y_test, y_pred),2))
pd.Series(y_pred).value_counts()
mat = confusion_matrix(y_test,y_pred,labels=['no','yes'])
print(mat)
y_test = label_binarize(y_test,classes=['no','yes'])
y_pred = label_binarize(y_pred,classes=['no','yes'])
print("Precision: ",round(precision_score(y_test,y_pred),2),"Recall: ",round(recall_score(y_test,y_pred),2))

In [ ]:
from sklearn.neural_network import MLPClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf_model=clf.fit(X_train_std, y_train)
y_pred = clf_model.predict(X_test_std)
print("Accuracy: ",round(accuracy_score(y_test, y_pred),2))
pd.Series(y_pred).value_counts()
mat = confusion_matrix(y_test,y_pred,labels=['no','yes'])
print(mat)
y_test = label_binarize(y_test,classes=['no','yes'])
y_pred = label_binarize(y_pred,classes=['no','yes'])
print("Precision: ",round(precision_score(y_test,y_pred),2),"Recall: ",round(recall_score(y_test,y_pred),2))

In [ ]:
import pixiedust
dataX=pixiedust.sampleData(1)

In [ ]:
dataX

In [ ]:
data_url = "https://data.cityofnewyork.us/api/views/e98g-f8hy/rows.csv?accessType=DOWNLOAD"
building_dataframe = pixiedust.sampleData(data_url,
forcePandas=True)

In [7]:
import pixiedust

#london_info = pixiedust.sampleData("https://files.datapress.com/london/dataset/london-borough-profiles/2015-09-24T15:50:01/London-boroughprofiles.zip")

Pixiedust database opened successfully


In [ ]:
#import bokeh

In [ ]:
#from bokeh.plotting import figure, show, output_file

In [ ]:
#a=figure()
#a.legend.location = "top_left"


In [10]:
display(data)

In [15]:
#pip install pixiegateway

In [ ]:
##################################################################################

In [ ]:
from pixiedust.display.app import *

@PixieApp
class GitHubTracking():
    @route()
    def main_screen(self):
        return """
<style>
    div.outer-wrapper {
        display: table;width:100%;height:300px;
    }
    div.inner-wrapper {
        display: table-cell;vertical-align: middle;height: 100%;width: 100%;
    }
</style>
<div class="outer-wrapper">
    <div class="inner-wrapper">
        <div class="col-sm-3"></div>
        <div class="input-group col-sm-6">
          <input id="query{{prefix}}" type="text" class="form-control" placeholder="Search projects on GitHub">
          <span class="input-group-btn">
            <button class="btn btn-default" type="button">Submit Query</button>
          </span>
        </div>
    </div>
</div>   


"""
app = GitHubTracking()
app.run()

In [10]:
import requests
import pandas
[[GitHubTracking]]
@route(query="*")
@templateArgs
def do_search(self, query):
    response = requests.get( "https://api.github.com/search/repositories?q={}".format(query))
    frames = [pandas.DataFrame(response.json()['items'])]
    while response.ok and "next" in response.links:
        response = requests.get(response.links['next']['url'])
        frames.append(pandas.DataFrame(response.json()['items']))

    pdf = pandas.concat(frames)
    response = requests.get( "https://api.github.com/search/repositories?q={}".format(query))
    if not response.ok:
        return "<div>An Error occurred: {{response.text}}</div>"
    return """<h1><center>{{pdf|length}} repositories were found</center></h1>"""

In [14]:
[[GitHubTracking]]
@route(query="*")
@templateArgs
def do_search(self, query):
    self.first_url = "https://api.github.com/search/repositories?q={}".format(query)
    self.prev_url = None
    self.next_url = None
    self.last_url = None
    
    response = requests.get(self.first_url)
    if not response.ok:
        return "<div>An Error occurred: {{response.text}}</div>"

    total_count = response.json()['total_count']
    self.next_url = response.links.get('next', {}).get('url', None)
    self.last_url = response.links.get('last', {}).get('url', None)
    return """
<h1><center>{{total_count}} repositories were found</center></h1>
<ul class="pagination">
  <li><a href="#" pd_options="page=first_url" pd_target="body{{prefix}}">First</a></li>
  <li><a href="#" pd_options="page=prev_url" pd_target="body{{prefix}}">Prev</a></li>
  <li><a href="#" pd_options="page=next_url" pd_target="body{{prefix}}">Next</a></li>
  <li><a href="#" pd_options="page=last_url" pd_target="body{{prefix}}">Last</a></li>
</ul>
<table class="table">
    <thead>
        <tr>
            <th>Repo Name</th>
            <th>Lastname</th>
            <th>URL</th>
            <th>Stars</th>
        </tr>
    </thead>
    <tbody id="body{{prefix}}">
        {{this.invoke_route(this.do_retrieve_page, page='first_url')}}
    </tbody>
</table>
"""

In [17]:
[[GitHubTracking]]
@route(page="*")
@templateArgs
def do_retrieve_page(self, page):
    url = getattr(self, page)
    if url is None:
        return "<div>No more rows</div>"
    response = requests.get(url)
    self.prev_url = response.links.get('prev', {}).get('url', None)
    self.next_url = response.links.get('next', {}).get('url', None)
    items = response.json()['items']
    return """
{%for row in items%}
<tr>
    <td>{{row['name']}}</td>
    <td>{{row.get('owner',{}).get('login', 'N/A')}}</td>
    <td><a href="{{row['html_url']}}" target="_blank">{{row['html_url']}}</a></td>
    <td>{{row['stargazers_count']}}</td>
</tr>
{%endfor%}
        """

In [21]:
@PixieApp
class RepoAnalysis():
    @route(analyse_repo_owner="*", analyse_repo_name="*")
    @templateArgs
    def do_analyse_repo(self, analyse_repo_owner, analyse_repo_name):
        self._analyse_repo_owner = analyse_repo_owner
        self._analyse_repo_name = analyse_repo_name
        return """
<div class="container-fluid">
    <div class="dropdown center-block col-sm-2">
      <button class="btn btn-primary dropdown-toggle" type="button" data-toggle="dropdown">
          Select Repo Data Set
          <span class="caret"></span>
      </button>
      <ul class="dropdown-menu" style="list-style:none;margin:0px;padding:0px">
          {%for analysis,_ in this.analyses%}
            <li>
                <a href="#" pd_options="analyse_type={{analysis}}" pd_target="analyse_vis{{prefix}}"
                    style="text-decoration: none;background-color:transparent">
                    {{analysis}}
                </a>
            </li>
          {%endfor%}
      </ul>
    </div> 
    <div id="analyse_vis{{prefix}}" class="col-sm-10"></div>
</div>
"""
app = RepoAnalysis()
app.run()